<a id='top'></a>

# Aspect and Percent North Covariates
Calculate aspect grid with the DEM used to create the flow network for each processing area (e.g. HUC8 in NHDPlus or
 region for BB, PWS, or Kodiak). We need the DEMs used for each flow direction grid, or we can't calculate covariates at
 the watershed scale.
 * **~~aspect_rch = calculate mean aspect for stream reach (zonal statistics)~~**
 * **~~aspect_cat = calculate mean aspect over catchments (zonal statistics)~~**
 * **north_wtd = create 1/0 grid of north facing cells from aspect grid (north = aspects from 315-45 degrees), ~~~use
  weighted flow accumulation to sum north facing cells, divide by flow accumulation grid to get % of north facing
  cells in each watershed~~~ Use tabulate area to quantify area of North cells (VALUE_1) and non-North (VALUE_0) cells
  for each watershed and calculate percent north as north_wtd = row[VALUE_1]/(row[VALUE_1]+row[VALUE_0])*100 .**

## Import modules

In [26]:
import arcpy
import os
import datetime
import time
import sys
arcpy.env.overwriteOutput = True
today = datetime.datetime.now()
# Make the time stamp.
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)

path = os.getcwd()
print (path)
print (sys.base_exec_prefix)

13102021
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\landcover
C:\Users\dwmerrigan\AppData\Local\Programs\ArcGIS\Pro\bin\Python\envs\arcgispro-py3


## Collect data and set working output locations
Set data_dir to folder containing all AKSSF regional subfolders/geoadatabases
* RS data folder: data_dir = r"W:\GIS\AKSSF"
* DM data folder: data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

In [27]:
# Set AKSSF Data directory
# data_dir = r"W:\GIS\AKSSF"

# dm local
data_dir = r"D:\GIS_Temp\AKSSF_BeckyCopy\AKSSF"

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces(workspace_type="Folder")
rois= []
print (regions)

# Path to create output folder/gdb
temppath = r"D:\GIS_temp" # Output folder
dirname = 'AKSSF_asp_met'
tempgdbname = 'AKSSF_asp_met.gdb'
temp_dir = os.path.join(temppath, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print('Working Folder already created...', temp_dir)

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print ('Output geodatabase created at', outcheck)
    outgdb = tempgdb.getOutput(0)

['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Bristol_Bay', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Cook_Inlet', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Copper_River', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Kodiak', 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Prince_William_Sound']
Working Folder already created... D:\GIS_temp\AKSSF_asp_met
Output location already exists D:\GIS_temp\AKSSF_asp_met\AKSSF_asp_met.gdb


In [28]:
# Set regional workspaces from AKSSF data folder and store in list
arcpy.env.workspace = data_dir
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Bristol_Bay',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Cook_Inlet',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Copper_River',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Kodiak',
 'D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Prince_William_Sound']

## Create aspect and north grids if they do not already exists
### NHDPlus

In [29]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with NHDPlus elev data

# elrasters = []
# nhd_dat = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_NHDPlus\\extracts"
# for dirpath, dirname, filenames in arcpy.da.Walk(nhd_dat, 'RasterDataset'):
#     for filename in filenames:
#         if filename == "elev_cm.tif":
#             elras = os.path.join(dirpath, filename)
#             elrasters.append(elras)
#             print (f'Appending {elras} to list')
#
# spatial_ref = arcpy.Describe(elrasters[0]).spatialReference
# print (spatial_ref.name)
# # Merge all nhdplus elevation rasters together
# elevname = 'AKSSF_NHDPlus_elev_cm.tif'
# AKSSF_elev_cm = arcpy.MosaicToNewRaster_management(elrasters,temp_dir,elevname, spatial_ref, "32_BIT_SIGNED", "5", "1", "LAST","FIRST")
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_NHDPlus_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_cm, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_NHDPlus_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))

Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020202\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020301\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020302\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020401\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020402\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020501\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSSF_NHDPlus\extracts\HRNHDPlusRasters19020502\elev_cm.tif to list
Appending C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\AKSS

### TauDEM - Create grids using Timms Composite DEM for entire TauDEM processed areas

In [31]:
# Code to create merged elevation grid, aspect and north aspect grids for entire AKSSF region with TauDEM elev data
# compDEM = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
# clip to study area
# AKSSF_elev_10m = r"D:\\GIS_temp\\AKSSF_Composite_10m_extract.tif"
#
# #make aspect raster for all of akssf from merged nhdplus
# from arcpy.sa import *
# aspname = 'AKSSF_Composite_10m_aspect.tif'
# asp_rast = Aspect(in_raster= AKSSF_elev_10m, method='Planar')
# asp_rast.save(os.path.join(temp_dir, aspname))
#
# # make north raster from aspect raster
# norname = 'AKSSF_Composite_10m_north_aspect.tif'
# aspect_in = Raster(asp_rast)
# north_raster = Con((aspect_in>=0)&(aspect_in<=45),1,Con((aspect_in<=360)&(aspect_in>=315),1,0))
# north_raster.save(os.path.join(temp_dir, norname))


## Begin GIS portion
- May be faster and save more space to create aspect rasters here using extract by mask?
- May be able to use aspect raster with tabulate intersection to calculate percent North (aspects from 315-45 degrees)

### <b>UPDATE - 2021-10-12 Do not run aspect metrics at this time. Only need to run percent North for watersheds</b>

In [43]:
from arcpy.sa import *
import re

# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")
regions

# Lists to store output tables
cat_asp_ztables = []
wtd_asp_ztables = []
cat_pernorth_taba_tables=[]
wtd_pernorth_taba_tables=[]

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Limit to cook inlet and bbay for testing
rois = ['Cook_Inlet', 'Bristol_Bay']
for roi in rois:
# Loop through regional folders
    for region in regions:
        print (region)
        if roi in str(region):
            print(f'{region} found for {roi}')
            if roi in nhdplus_dat:
                # NHDPLus raster data for DM pc, variables will be overwritten below if they exist in source folder.
                asp_rast = r"D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_aspect.tif"
                elev_rast = r"D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_elev_cm.tif"
                nor_rast = r"D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_north_aspect.tif"
                print(f'NHD data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            elif roi in tauDem_dat:
                # TauDEM raster data for DM pc, variables will be overwritten below if they exist in source folder.
                nor_rast = r"D:\\GIS_temp\\AKSSF_asp_met\\AKSSF_Composite_10m_north_aspect.tif"
                asp_rast = r"D:\\GIS_temp\\AKSSF_asp_met\\AKSSF_Composite_10m_aspect.tif"
                elev_rast = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\hydrography\data\topography\AKSSF_Elevation_Composite_10m.tif"
                print(f'TauDem data: north raster - {nor_rast}')
                print(f'         aspect raster - {asp_rast}')
                print(f'      elevation raster - {elev_rast}')
                print('----------')

            stats_fields = ['MEAN','MAX','MIN','STD']
            wtds = []
            cats = []
            # Start iter timing function
            iteration_start = time.time()
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
            for dirpath, dirnames, filenames in walk:
                for filename in filenames:
                    # Create list of watersheds
                    if "wtd" in filename:
                        wtds.append(os.path.join(dirpath, filename))
                        outgdb = gdb

                    # # Select aspect raster
                    # elif 'aspect' in filename:
                    #     asp_rast = os.path.join(dirpath, filename)

                    # Select north raster
                    elif 'north_asp' in filename:
                        nor_rast = os.path.join(dirpath, filename)

                    # Select catch_int fc (catchments of interest for region) and make a copy
                    elif 'cats_intersect' in filename:
                        cats = arcpy.FeatureClassToFeatureClass_conversion(os.path.join(dirpath,filename),
                                                                           'memory\\','cat_copy')
                        arcpy.AddField_management(cats,"NHDPlusID_txt",field_type='TEXT')
                        # Copy NHDPlusID as text field for use in zonal stats
                        with arcpy.da.UpdateCursor(cats, ['NHDPlusID','NHDPlusID_txt']) as cur:
                            for row in cur:
                                row[1] = roi + "_" + str(row[0])
                                # Update rows
                                cur.updateRow(row)
                            del(row)
                        del(cur)

            print (f'Calculating aspect metrics and percent north for catchments & watersheds of interest in {roi}'
                   f' region')
            print ('----------')
            print(f'Geodatabase: {gdb}')
            print ('----------')
            print (f'Elevation Raster: {arcpy.Describe(asp_rast).baseName}')
            print ('----------')
            print (f'{len(wtds)} Watersheds to process')
            print ('----------')
            print (f'Catchment intersect {cats} selected')

            try:
                mergestart = time.time()
                # Set output names/paths for watershed and catchment tables
                wtd_merge_name = roi + "_Watersheds_Merge"
                wtd_merge_path = os.path.join(outgdb,wtd_merge_name)
                # # Aspect variables
                # wtd_merge_asp_table_name = roi + "_Watersheds_Merge_AspectZstats"
                # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
                # cat_asp_table_name = roi + "_Catchments_AspectZstats"
                # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)
                # Percent North variables
                wtd_merge_pernorth_table_name = roi + "_Watersheds_Merge_PercentNorth"
                wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
                # cat_pernorth_table_name = roi + "_Catchments_PercentNorth"
                # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

                # Merge watersheds
                wtd_merge = arcpy.Merge_management( wtds, wtd_merge_path ,add_source='ADD_SOURCE_INFO')
                # Add wtd_id field
                arcpy.AddField_management(wtd_merge,'wtd_id',field_type='TEXT')
                # Add region field
                arcpy.AddField_management(wtd_merge,'region',field_type='TEXT')
                # Populate watershed id and region information using update cursor - faster than field calc
                with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','wtd_id','region']) as cur:
                    for row in cur:
                        row[1] = re.findall('\d+', row[0])[0]
                        row[2] = roi
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)
                mergestop = time.time()
                mergetime = int (mergestop - mergestart)
                print(f'Watershed Merge for {roi} Elapsed time: ({datetime.timedelta(seconds=mergetime)})')
                print('----------')

                # Begin Zonal Stats/Tabulate Area
                zstat_start = time.time()
                print(f'Begin zonal statistics min/mean/max std dev for watersheds and catchments in {roi} region')

                # # Aspect Zonal statistics  for watersheds
                # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="wtd_id",
                #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
                #                                                statistics_type='ALL')
                # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
                # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
                # wtd_asp_ztables.append(wtd_asp_metrics_table)

                # # Aspect Zonal statistics for catchments
                # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="NHDPlusID_txt",
                #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
                #                                                statistics_type='ALL')
                # cat_asp_ztables.append(cat_asp_metrics_table)
                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
                print('----------')

                # Tabulate Area with north grid and catchments/watersheds
                tabarea_start = time.time()
                print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
                # Percent North Tabulate area for watersheds
                wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge, zone_field='wtd_id',
                                                            in_class_data=nor_rast, class_field="Value",
                                                            out_table=wtd_merge_pernorth_table_path)

                arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
                arcpy.AddField_management(wtd_per_north_tabarea, 'north_wtd', field_type='Float')

                with arcpy.da.UpdateCursor(wtd_per_north_tabarea,['wtd_id','region', 'VALUE_0',
                                                                        'VALUE_1', 'north_wtd']) as cur:
                    for row in cur:
                        row[1] = roi
                        row[4] = row[3]/(row[3]+row[2])*100
                        # Update
                        cur.updateRow(row)
                    del(row)
                del(cur)

                wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

                # # Percent North Tabulate Area for catchments
                # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='NHDPlusID',
                #                                             in_class_data=nor_rast,"Value",
                #                                             out_table=cat_pernorth_table_path)

                # # Add and calculate region identifier field for catchment table
                # arcpy.AddField_management(cat_per_north_tabarea, 'wtd_id', field_type='Float')
                # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
                # arcpy.AddField_management(cat_per_north_tabarea, 'north_cat', field_type='Float')
                #
                # with arcpy.da.UpdateCursor(cat_per_north_tabarea,['NHDPlusID','region', 'VALUE_0',
                #                                                         'VALUE_1', 'north_cat']) as cur:
                #     for row in cur:
                #         row[1] = roi
                #         row[4] = row[3]/(row[3]+row[2])*100
                #         # Update
                #         cur.updateRow(row)
                #     del(row)
                # del(cur)
                # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
                tabarea_stop = time.time()
                tabarea_time = int (tabarea_stop - tabarea_start)
                print(f'Tabulate area for {roi} Elapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
                print('----------')

            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

    else:
        print(f'Region {str(roi)} not found in {region}')
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Bristol_Bay
D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Cook_Inlet
D:\GIS_temp\AKSSF_BeckyCopy\AKSSF\Cook_Inlet found for Cook_Inlet
NHD data: north raster - D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_north_aspect.tif
         aspect raster - D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_aspect.tif
      elevation raster - D:\GIS_temp\AKSSF_asp_met\AKSSF_NHDPlus_elev_cm.tif
----------
Calculating aspect metrics and percent north for catchments & watersheds of interest in Cook_Inlet region
----------
Geodatabase: ['D:\\GIS_temp\\AKSSF_BeckyCopy\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb']
----------
Elevation Raster: AKSSF_NHDPlus_aspect
----------
114 Watersheds to process
----------
Watershed Merge for Cook_Inlet Elapsed time: (0:00:22)
----------
Begin zonal statistics min/mean/max std dev for watersheds and catchments in Cook_Inlet region
Zonal Stats for Cook_Inlet Elapsed time: (0:00:00)
----------
Begin tabulate area of north facing cells for watersheds and catchments in Co

## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [ ]:
# Merge tables together
        # wtd_per_north = arcpy.Merge_management(wtd_elev_ztables, wtd_per_north_table_out)
        # cat_per_north = arcpy.Merge_management(cat_asp_ztables, cat_per_north_table_out)
        # Use update cursor to calculate percent North
                        # with arcpy.da.UpdateCursor(wtd_per_north,['VALUE_0','VALUE_1','wtd_aspect_mn']) as cur:
                        #     for row in cur:
                        #         row[2] = row[1]/(row[0] + row[1]) *100
                        #         cur.updateRow(row)
                        #     del(row)
                        # del(cur)

# Select tables and rename fields

# Export to csv
